# Clasificación de estado de presión de llantas vehículos a partir de imágenes

###Juan Nicolás Soto Rios
###Katherine Xiomar González Santacruz
###Frank M. Córdoba O.


# 1. Cargando Archivos

In [ ]:
import os
os.getcwd()
import cv2
from google.colab.patches import cv2_imshow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
flat_class = []
for image in os.listdir('flat.class'):
  flat_class.append(cv2.imread(os.path.join('flat.class', image)))

In [ ]:
full_class = []
for image in os.listdir('full.class'):
  full_class.append(cv2.imread(os.path.join('full.class', image)))

In [ ]:
cv2_imshow(flat_class[0]), cv2_imshow(full_class[0]) 

In [ ]:
print(flat_class[0].shape)
print(len(flat_class))

print(full_class[0].shape)
print(len(full_class))

In [ ]:
flat_gray = []
full_gray = []
full_data = []
for img in flat_class:
  flat_gray.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())
  full_data.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())
for img in full_class:
  full_gray.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())
  full_data.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)).flatten())

flat_gray = np.array(flat_gray)
full_gray = np.array(full_gray)
full_data = np.array(full_data)

In [ ]:
print(flat_gray.shape)
print(len(flat_gray))

print(full_gray.shape)
print(len(full_gray))

print(full_data.shape)
print(len(full_data))

In [ ]:
etiquetas = np.zeros((300,))
etiquetas = np.append(etiquetas, np.ones((300,)))
etiquetas.shape

# 2. Analizar dimensionalidad de los datos

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=200)

pca.fit(full_data)

In [ ]:
print(pca.explained_variance_ratio_[:5])
print(pca.explained_variance_ratio_.sum())

In [ ]:
# Plot
c = []
v_ac = []
v_i = 0
for v in pca.explained_variance_ratio_:
  v_i += v
  v_ac.append(v_i) 
plt.plot(range(len(pca.explained_variance_ratio_)), v_ac)
plt.ylabel('Varianza Explicada')
plt.xlabel('Componentes Principales')
plt.title('Varianza Explicada Acumulada')
plt.show()

In [ ]:
n_c = 0
i = 0
counting = True
while counting:
  n_c += pca.explained_variance_ratio_[i]
  i += 1
  if n_c >= 0.8:
    print(f'Numero de variables: {i}')
    counting = False
  

In [ ]:
pca = PCA(3) 
projected = pca.fit_transform(full_data)

plt.scatter(projected[:, 0], projected[:, 1],
            c=etiquetas, edgecolor='none', alpha=0.5, cmap=plt.cm.get_cmap('viridis', 2))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

In [ ]:
import plotly.express as px
fig = px.scatter_3d( x= projected[:, 0], y= projected[:, 1], z= projected[:, 2],
              color= etiquetas, )
fig.show()

In [ ]:
pca = PCA(70)

X = pca.fit_transform(full_data)
y = etiquetas

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test_p, y_train, y_test_p = train_test_split(X, y, test_size=0.3, random_state=42)


print(X_train.shape)
print(X_test_p.shape)
print(y_train.shape)
print(y_test_p.shape)

# 3. Red Neuronal

# 3.1 Modelo Red Neuronal 1

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

X_val, X_test, y_val, y_test = train_test_split(X_test_p, y_test_p, test_size= 0.3, random_state= 42)

validation = True

n_features = X_train.shape[1]
# Definicion del modelo
nn_model = Sequential()
nn_model.add(Dense(128, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
nn_model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
nn_model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
nn_model.add(Dense(1, activation='sigmoid'))

# Callback
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# Compilacion del modelo
nn_model.compile(optimizer='adam', loss= tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
start = time.time()

# fit
if validation: 
  nn_model.fit(X_train, y_train, epochs=20, batch_size=10,  callbacks=[callback],  validation_data = (X_val, y_val))
else:
   nn_model.fit(X_train, y_train, epochs=20, batch_size=10,  callbacks=[callback],  validation_data = None)
# evaluacion
if validation:
  # train accuracy
  loss, acc = nn_model.evaluate(X_train, y_train, verbose=0)
  print('Train Accuracy: %.3f' % acc)
  # test acccuracy
  loss, acc = nn_model.evaluate(X_test, y_test, verbose=0)
  print('Test Accuracy: %.3f' % acc)
else:
  # train accuracy
  loss, acc = nn_model.evaluate(X_train, y_train, verbose=0)
  print('Train Accuracy: %.3f' % acc)
  # test acccuracy
  loss, acc = nn_model.evaluate(X_test_p, y_test_p, verbose=0)
  print('Test Accuracy: %.3f' % acc)

end = time.time()
t_total = (end-start)/60
print(f'Tiempo de entrenamiento: {t_total} minutos')

In [ ]:
# Diagrama del modelo
from keras.utils.vis_utils import plot_model
plot_model(nn_model,  show_shapes=True, show_layer_names=True)

In [ ]:
# Predicciones
y_pred_nn = nn_model.predict(X_test_p)
y_pred_nn = np.array(y_pred_nn).flatten()
y_pred_nn = (y_pred_nn > 0.9).astype(int)
print(classification_report(y_test_p, y_pred_nn))

In [ ]:
# Matriz de confusion
cm = confusion_matrix(y_test_p, y_pred_nn)
print(cm)

In [ ]:
# Matriz de confusion
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels= None)


disp = disp.plot(include_values= True, cmap='viridis')

plt.show()

# 3.2 Modelo Red Neuronal 2

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense


validation = True

n_features = X_train.shape[1]
# Definicion del modelo
nn_model = Sequential()
nn_model.add(Dense(254, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
nn_model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
nn_model.add(Dense(64, activation='relu', kernel_initializer='he_normal'))
nn_model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
nn_model.add(Dense(1, activation='sigmoid'))

# Callback
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# Compilacion del modelo
nn_model.compile(optimizer='adam', loss= tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
start = time.time()
# fit
if validation: 
  nn_model.fit(X_train, y_train, epochs=20, batch_size=10,  callbacks=[callback],  validation_data = (X_val, y_val))
else:
   nn_model.fit(X_train, y_train, epochs=20, batch_size=10,  callbacks=[callback],  validation_data = None)
# evaluacion
if validation:
  # train accuracy
  loss, acc = nn_model.evaluate(X_train, y_train, verbose=0)
  print('Train Accuracy: %.3f' % acc)
  # test acccuracy
  loss, acc = nn_model.evaluate(X_test, y_test, verbose=0)
  print('Test Accuracy: %.3f' % acc)
else:
  # train accuracy
  loss, acc = nn_model.evaluate(X_train, y_train, verbose=0)
  print('Train Accuracy: %.3f' % acc)
  # test acccuracy
  loss, acc = nn_model.evaluate(X_test_p, y_test_p, verbose=0)
  print('Test Accuracy: %.3f' % acc)
end = time.time()
t_total = (end-start)/60
print(f'Tiempo de entrenamiento: {t_total} minutos')

In [ ]:
# Diagrama del modelo
from keras.utils.vis_utils import plot_model
plot_model(nn_model,  show_shapes=True, show_layer_names=True)

In [ ]:
# Predicciones
y_pred_nn = nn_model.predict(X_test_p)
y_pred_nn = np.array(y_pred_nn).flatten()
y_pred_nn = (y_pred_nn > 0.9).astype(int)
print(classification_report(y_test_p, y_pred_nn))

In [ ]:
# Matriz de confusion
cm = confusion_matrix(y_test_p, y_pred_nn)
print(cm)

In [ ]:
# Matriz de confusion
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels= None)


disp = disp.plot(include_values= True, cmap='viridis')

plt.show()

# 4. Redes Convolucionales

## 4.1 Preparacion de Imagenes

In [ ]:
full_data_2 = []
for img in flat_class:
  full_data_2.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)))

for img in full_class:
  full_data_2.append(np.array(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)))

In [ ]:
## Particion de la base de datos en train - test
from sklearn.model_selection import train_test_split
X_train_i, X_test_p_i, y_train_i, y_test_p_i = train_test_split(np.array(full_data_2), etiquetas, test_size=0.3, random_state=42)


print(X_train_i.shape)
print(X_test_p_i.shape)
print(y_train_i.shape)
print(y_test_p_i.shape)

In [ ]:
# Particion de datos para validacion y test
X_val, X_test, y_val, y_test = train_test_split(X_test_p_i, y_test_p_i, test_size=0.3, random_state=42)

## 4.1 Modelo Red Convolucional 1

In [ ]:
# Definicion del modelo
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding= "same" , input_shape=(240, 240,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding= "same", activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding= "same", activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

In [ ]:
# Resumen del modelo
model.summary()

In [ ]:
# Diagrama del modelo
from keras.utils.vis_utils import plot_model
plot_model(model,  show_shapes=True, show_layer_names=True)

In [ ]:
# Se establece Callback
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

In [ ]:
# Entrenamiento
start = time.time()
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train_i, y_train_i, epochs=10, 
                    validation_data=(X_val, y_val), batch_size =32, callbacks = [callback])
end = time.time()
t_total = (end-start)/60
print(f'Tiempo de entrenamiento: {t_total} minutos')

In [ ]:
# Historico de metricas durante entrenamiento
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

In [ ]:
print(test_acc)

In [ ]:
# Classification report
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, plot_confusion_matrix
y_pred = model.predict(X_test)
y_pred= np.array(y_pred).flatten()
y_pred_nn = (y_pred> 0.9).astype(int)
print(classification_report(y_test, y_pred_nn))

In [ ]:
# Matriz de confusion
cm = confusion_matrix(y_test, y_pred_nn)
print(cm)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels= None)


disp = disp.plot(include_values= True, cmap='viridis')

plt.show()

## 4.3 Modelo Red Convolucional 2

In [ ]:
# Definicion del modelo
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding= "same" , input_shape=(240, 240,1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding= "same", activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), padding= "same", activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), padding= "same", activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1))

In [ ]:
# Resumen del modelo
model.summary()

In [ ]:
# Diagrama del modelo
from keras.utils.vis_utils import plot_model
plot_model(model,  show_shapes=True, show_layer_names=True)

In [ ]:
# Se establece Callback
callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=3, verbose=0,
    mode='auto', baseline=None, restore_best_weights=False
)

In [ ]:
# Entrenamiento
start = time.time()
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train_i, y_train_i, epochs=10, 
                    validation_data=(X_val, y_val), batch_size =32, callbacks = [callback])
end = time.time()
t_total = (end-start)/60
print(f'Tiempo de entrenamiento: {t_total} minutos')

In [ ]:
# Historico de metricas durante entrenamiento
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

In [ ]:
# Classification report
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, plot_confusion_matrix
y_pred = model.predict(X_test)
y_pred= np.array(y_pred).flatten()
y_pred_nn = (y_pred> 0.9).astype(int)
print(classification_report(y_test, y_pred_nn))

In [ ]:
# Matriz de confusion
cm = confusion_matrix(y_test, y_pred_nn)
print(cm)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels= None)


disp = disp.plot(include_values= True, cmap='viridis')

plt.show()